# Автонейминг 

In [ ]:
import os
from datetime import datetime

def rename_files(folder_path):
    for letter_dir in os.listdir(folder_path):
        letter_path = os.path.join(folder_path, letter_dir)
        
        if os.path.isdir(letter_path):
            for i, filename in enumerate(os.listdir(letter_path), 1):
                new_name = f"{letter_dir}_{i:03d}_raw_{datetime.now().strftime('%Y%m%d')}.tiff"
                os.rename(
                    os.path.join(letter_path, filename),
                    os.path.join(letter_path, new_name)
                )

rename_files("путь/к/папке/с/буквами")

# UUID-имена

In [ ]:
import uuid
filename = f"{letter}_{str(uuid.uuid4())[:8]}.png"

# Хеш-имена

In [ ]:
import hashlib
with open(image_path, "rb") as f:
    file_hash = hashlib.md5(f.read()).hexdigest()[:6]
filename = f"{letter}_{file_hash}.png"

# Ч/Б + адаптивная бинаризация

In [ ]:
!pip install opencv-python

In [ ]:
!pip install tqdm

In [ ]:
import cv2
import os
import numpy as np
from tqdm import tqdm

def convert_to_bw(input_dir, output_dir, threshold=180):
    """
    Конвертирует все изображения в папке в чёрно-белые с адаптивной бинаризацией
    :param input_dir: Папка с исходными изображениями
    :param output_dir: Папка для результатов
    :param threshold: Порог бинаризации (0-255)
    """
    os.makedirs(output_dir, exist_ok=True)
    
    for root, _, files in os.walk(input_dir):
        for file in tqdm(files, desc="Обработка изображений"):
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff')):
                # Создаем аналогичную структуру папок
                rel_path = os.path.relpath(root, input_dir)
                os.makedirs(os.path.join(output_dir, rel_path), exist_ok=True)
                
                # Обработка изображения
                img_path = os.path.join(root, file)
                img = cv2.imread(img_path)
                
                # Конвертация в grayscale
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
                # Адаптивная бинаризация
                bw = cv2.adaptiveThreshold(
                    gray, 255,
                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                    cv2.THRESH_BINARY_INV,
                    blockSize=21,
                    C=7
                )
                
                output_path = os.path.join(output_dir, rel_path, file)
                cv2.imwrite(output_path, bw)

input_folder = "data/data_1_test/raw/"  # Папка с исходными изображениями
output_folder = "data/data_1_test/processed/"  # Папка для чёрно-белых версий

convert_to_bw(input_folder, output_folder)

In [ ]:
import cv2
import os
import numpy as np
from tqdm import tqdm

def enhance_image(img):
    """
    Улучшение качества изображения перед бинаризацией
    :param img: Входное изображение (grayscale)
    :return: Улучшенное изображение
    """
    # Удаление шумов (Non-Local Means Denoising)
    denoised = cv2.fastNlMeansDenoising(img, h=15, templateWindowSize=7, searchWindowSize=21)
    
    # Улучшение контраста (CLAHE)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    contrast = clahe.apply(denoised)
    
    # Морфологическое закрытие для соединения разрывов
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    closed = cv2.morphologyEx(contrast, cv2.MORPH_CLOSE, kernel)
    
    return closed

def convert_to_bw(input_dir, output_dir, threshold=180):
    """
    Конвертирует все изображения в папке в чёрно-белые с улучшением качества
    :param input_dir: Папка с исходными изображениями
    :param output_dir: Папка для результатов
    :param threshold: Порог бинаризации (0-255)
    """
    os.makedirs(output_dir, exist_ok=True)
    
    for root, _, files in os.walk(input_dir):
        for file in tqdm(files, desc="Обработка изображений"):
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff')):
                # Создаем аналогичную структуру папок
                rel_path = os.path.relpath(root, input_dir)
                os.makedirs(os.path.join(output_dir, rel_path), exist_ok=True)
                
                # Обработка изображения
                img_path = os.path.join(root, file)
                img = cv2.imread(img_path)
                
                # Конвертация в grayscale
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
                # Улучшение качества
                enhanced = enhance_image(gray)
                
                # Адаптивная бинаризация
                bw = cv2.adaptiveThreshold(
                    enhanced, 255,
                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                    cv2.THRESH_BINARY_INV,
                    blockSize=21,
                    C=7
                )
                
                # Дополнительная очистка мелких артефактов
                kernel_clean = np.ones((2,2), np.uint8)
                cleaned = cv2.morphologyEx(bw, cv2.MORPH_OPEN, kernel_clean)
                
                output_path = os.path.join(output_dir, rel_path, file)
                cv2.imwrite(output_path, cleaned)

# Параметры обработки
input_folder = "data/data_1_test/raw/"  # Папка с исходными изображениями
output_folder = "data/data_1_test/processed/"  # Папка для чёрно-белых версий

convert_to_bw(input_folder, output_folder)